In [1]:
'''
DATASET :  https://www.kaggle.com/datasets/devavratatripathy/ecg-dataset
'''

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import InputLayer,Dense
from tensorflow.keras.models import Sequential

In [2]:
ecg_dataset = pd.read_csv('ecg.csv')
X = ecg_dataset.values  
y = np.zeros(X.shape[0])  

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
input_dim, encoding_dim = X_train.shape[1], 16
encoder = Sequential([
    InputLayer(input_shape=(input_dim,)),
    Dense(32, activation='relu'),
    Dense(encoding_dim, activation='relu')
])

In [6]:

decoder = Sequential([
    InputLayer(input_shape=(encoding_dim,)),
    Dense(32, activation='relu'),
    Dense(input_dim, activation='sigmoid')
])


In [7]:
autoencoder = Sequential([encoder, decoder])

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

autoencoder.summary()

Epoch 1/10
125/125 [==============================] - 1s 5ms/step - loss: 0.9369 - val_loss: 0.8404
Epoch 2/10
125/125 [==============================] - 0s 3ms/step - loss: 0.7302 - val_loss: 0.7791
Epoch 3/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6946 - val_loss: 0.7583
Epoch 4/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6795 - val_loss: 0.7463
Epoch 5/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6714 - val_loss: 0.7401
Epoch 6/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6661 - val_loss: 0.7361
Epoch 7/10
125/125 [==============================] - 1s 7ms/step - loss: 0.6628 - val_loss: 0.7333
Epoch 8/10
125/125 [==============================] - 1s 7ms/step - loss: 0.6590 - val_loss: 0.7290
Epoch 9/10
125/125 [==============================] - 0s 4ms/step - loss: 0.6557 - val_loss: 0.7256
Epoch 10/10
125/125 [==============================] - 1s 7ms/step - loss: 0.6529 - val_loss: 0.7229

In [8]:
predictions = autoencoder.predict(X_test)

mse = np.mean((X_test - predictions)**2, axis=1)

threshold = np.percentile(mse, 95)

y_test_pred = np.where(mse > threshold, 1, 0)

32/32 [==============================] - 0s 1ms/step


In [9]:
accuracy = accuracy_score(y_test, y_test_pred)
accuracy_score(y_test,y_test_pred)

0.95